In [1]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json
from openai import OpenAI

In [6]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        client = OpenAI(base_url="https://api.fanar.qa/v1",api_key=os.environ["FANAR_API_KEY"],)
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="azure"
deployment = APIKeyManager(model_type, "./azure.env")


In [ ]:
class TopicClassifier:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def classify_topic(self, topic, information):
        """
        Classify podcast topic and determine optimal approach
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            
        Returns:
            JSON with classification results
        """
        
        prompt = f"""
أنت خبير في تحليل المواضيع وتصنيفها لإنتاج بودكاست عربي طبيعي وجذاب.

المهمة: حلل الموضوع التالي وحدد أفضل نهج لتقديمه في بودكاست عربي.

الموضوع: {topic}
المعلومات الأساسية: {information}

قم بتحليل الموضوع وإرجاع النتيجة بصيغة JSON تحتوي على:

{{
    "primary_category": "الفئة الرئيسية",
    "category_justification": "سبب اختيار هذه الفئة بناءً على طبيعة الموضوع",
    "secondary_tags": ["تصنيفات فرعية مرتبطة"],
    "optimal_style": "الأسلوب الأمثل للمناقشة",
    "emotional_intent": "القصد العاطفي للحلقة",
    "discourse_pattern": "نمط الخطاب المناسب",
    "audience_engagement_goal": "هدف تفاعل الجمهور",
    "cultural_sensitivity_level": "مستوى الحساسية الثقافية",
    "controversy_potential": "احتمالية الجدل",
    "key_discussion_angles": [
        "زوايا النقاش الرئيسية المتوقعة",
        "النقاط التي ستثير اهتمام الجمهور العربي"
    ],
    "natural_tension_points": [
        "نقاط التوتر الطبيعية في الموضوع",
        "الجوانب التي قد تثير جدلاً صحياً"
    ],
    "cultural_connection_opportunities": [
        "فرص الربط بالثقافة العربية",
        "المراجع المحلية والإقليمية ذات الصلة"
    ]
}}

الفئات المتاحة:
1. "العلوم والتكنولوجيا" - للمواضيع التقنية والعلمية والابتكارات
2. "السياسة والشؤون العامة" - للمواضيع السياسية والأحداث الجارية والقضايا العامة
3. "القضايا الاجتماعية" - للمواضيع المجتمعية والعلاقات والقيم والتحديات الاجتماعية
4. "الرياضة والترفيه" - للمواضيع الرياضية والفنية والترفيهية
5. "التاريخ والثقافة" - للمواضيع التاريخية والتراثية والثقافية

الأساليب المتاحة:
- "حواري" - حوار طبيعي وودي بين المقدم والضيف
- "تعليمي" - تركيز على الشرح والتعليم بطريقة ممتعة
- "ترفيهي" - مرح وخفيف مع لمسات فكاهية
- "تحليلي" - نقاش عميق ومتخصص وتحليلي

أنماط الخطاب:
- "رسمي" - لغة رسمية ومحترمة
- "ودي" - لغة دافئة ومألوفة
- "جدلي" - نقاش حيوي مع وجهات نظر متعددة
- "سردي" - أسلوب حكواتي وقصصي

القصد العاطفي:
- "تعليمي" - إكساب المعرفة والفهم
- "ملهم" - تحفيز وإلهام المستمعين
- "ترفيهي" - إمتاع وتسلية
- "مثير للجدل" - إثارة النقاش والتفكير النقدي
- "مطمئن" - تهدئة المخاوف وتقديم الراحة النفسية

مستوى الحساسية الثقافية:
- "عالي" - يتطلب حذراً شديداً في التعامل
- "متوسط" - يحتاج مراعاة ثقافية معتدلة  
- "منخفض" - موضوع مقبول عموماً

احتمالية الجدل:
- "عالية" - موضوع مثير للجدل بطبيعته
- "متوسطة" - قد يثير بعض الاختلافات
- "منخفضة" - موضوع مقبول عموماً

تعليمات مهمة:
- حلل الموضوع بعمق وليس بشكل سطحي
- اعتبر السياق الثقافي العربي في التحليل
- ركز على ما يجعل الموضوع جذاباً للجمهور العربي
- تأكد أن التصنيف يخدم إنتاج محتوى طبيعي وتلقائي
- لا تضع علامات ```json في البداية أو النهاية
- أرجع JSON صحيح فقط بدون أي نص إضافي
-المدة 10 دقائق هي المدة المثلى للحلقة
"""

        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "أنت خبير في تحليل المواضيع وتصنيفها لإنتاج بودكاست عربي احترافي. تخصصك في فهم الجمهور العربي واهتماماته."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3  # Lower temperature for more consistent classification
        )
        
        return response.choices[0].message.content

    def validate_classification(self, classification_json):
        """
        Validate that the classification contains all required elements
        """
        required_keys = [
            "primary_category", "category_justification", "secondary_tags",
            "optimal_style", "emotional_intent", "discourse_pattern",
            "audience_engagement_goal", "cultural_sensitivity_level",
            "controversy_potential", "recommended_duration",
            "key_discussion_angles", "natural_tension_points",
            "cultural_connection_opportunities"
        ]
        
        try:
            import json
            classification = json.loads(classification_json)
            
            missing_keys = []
            for key in required_keys:
                if key not in classification:
                    missing_keys.append(key)
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Classification validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"




# Alternative test topics to try:
test_topics = {
    "technology": {
        "topic": "الذكاء الاصطناعي وتأثيره على سوق العمل في الخليج",
        "info": "تشهد دول الخليج نمواً سريعاً في تطبيقات الذكاء الاصطناعي، مما يثير تساؤلات حول مستقبل الوظائف"
    },
    "politics": {
        "topic": "العلاقات العربية الإيرانية في المنطقة",
        "info": "تطورات الوضع الجيوسياسي في منطقة الخليج وتأثيرها على الاستقرار الإقليمي"
    },
    "sports": {
        "topic": "كأس العالم قطر 2022 وإرث البطولة",
        "info": "تقييم تأثير استضافة قطر لكأس العالم على التطوير الرياضي والسياحة في المنطقة"
    },
    "history": {
        "topic": "طريق الحرير القديم ودوره في التجارة العربية",
        "info": "استكشاف كيف ساهمت الطرق التجارية القديمة في ازدهار الحضارة العربية والتبادل الثقافي"
    }
}

def test_multiple_topics(classifier):
    """Test classifier with different topic types"""
    for category, data in test_topics.items():
        print(f"\n{'='*50}")
        print(f"Testing {category.upper()} topic:")
        print(f"Topic: {data['topic']}")
        print(f"{'='*50}")
        
        result = classifier.classify_topic(data['topic'], data['info'])
        
        try:
            parsed = json.loads(result)
            print(f"Classified as: {parsed['primary_category']}")
            print(f"Style: {parsed['optimal_style']}")
            print(f"Intent: {parsed['emotional_intent']}")
        except:
            print("Classification result:", result)
        
        print("\n")

In [8]:
# Testing Instructions:

# To test Step 1, add this to a new cell in your notebook:

# Test Step 1: Topic Classification
classifier = TopicClassifier(deployment, "Fanar")

# Test with the singlehood topic
topic = "ظاهرة العنوسة في المجتمع العربي: أسباب وحلول"
information = '''
ظاهرة العنوسة تشهد ارتفاعاً ملحوظاً في المجتمعات العربية، حيث تشير الإحصائيات إلى أن 30% من النساء في دول الخليج 
و 25% من الرجال تجاوزوا سن الثلاثين دون زواج، مقارنة بـ 15% قبل عقدين. 
الأسباب متعددة تشمل ارتفاع تكاليف الزواج والسكن، تغير الأولويات المهنية والتعليمية، صعوبة التوافق بين الشريكين، 
وتأثير وسائل التواصل الاجتماعي على توقعات الشباب من الزواج.
'''

# Run classification
classification_result = classifier.classify_topic(topic, information)
print("Classification Result:")
print(classification_result)

# Validate the result
is_valid, message = classifier.validate_classification(classification_result)
print(f"\nValidation: {message}")

# Parse and examine the JSON
import json
try:
    parsed_result = json.loads(classification_result)
    print(f"\nPrimary Category: {parsed_result['primary_category']}")
    print(f"Optimal Style: {parsed_result['optimal_style']}")
    print(f"Emotional Intent: {parsed_result['emotional_intent']}")
    print(f"Discourse Pattern: {parsed_result['discourse_pattern']}")
except:
    print("Error parsing JSON result")

Classification Result:
{
    "primary_category": "القضايا الاجتماعية",
    "category_justification": "الموضوع يتناول قضية اجتماعية بارزة تؤثر على العلاقات والقيم في المجتمعات العربية، مما يجعله مناسباً لهذه الفئة.",
    "secondary_tags": ["العلاقات الزوجية", "الشباب العربي", "التحديات الاقتصادية", "التغيرات الثقافية"],
    "optimal_style": "تحليلي",
    "emotional_intent": "ملهم",
    "discourse_pattern": "ودي",
    "audience_engagement_goal": "تشجيع النقاش المفتوح وتقديم حلول عملية يمكن أن يتفاعل معها الجمهور.",
    "cultural_sensitivity_level": "عالي",
    "controversy_potential": "متوسطة",
    "recommended_duration": "30-40 دقيقة",
    "key_discussion_angles": [
        "الأسباب الاقتصادية والاجتماعية وراء ارتفاع معدلات العنوسة في العالم العربي.",
        "دور وسائل التواصل الاجتماعي في تشكيل توقعات الشباب من الزواج.",
        "كيف يمكن للمجتمع العربي أن يتعامل مع هذه الظاهرة بطريقة بناءة؟",
        "قصص نجاح لأفراد تغلبوا على تحديات العنوسة وحققوا التوازن في حياتهم."
    ],
    "na

In [9]:
class SimplePersonaGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_personas(self, topic, information, classification_result):
        """
        Generate simple but effective host and guest personas
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            
        Returns:
            JSON with simple host and guest personas
        """
        
        # Parse classification to understand the requirements
        import json
        try:
            classification = json.loads(classification_result)
        except:
            raise ValueError("Invalid classification JSON provided")
        
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        
        prompt = f"""
أنت خبير في تصميم شخصيات البودكاست العربي.

المهمة: أنشئ مقدم وضيف بسيطين ومناسبين لهذا الموضوع.

الموضوع: {topic}
المعلومات: {information}
الفئة: {primary_category}
الأسلوب المطلوب: {optimal_style}

أرجع النتيجة بصيغة JSON بسيط:

{{
    "host": {{
        "name": "اسم المقدم",
        "age": عمر رقمي,
        "background": "خلفية مختصرة في جملة واحدة",
        "personality": "وصف شخصيته في جملة واحدة",
        "speaking_style": "أسلوب حديثه في جملة واحدة"
    }},
    "guest": {{
        "name": "اسم الضيف", 
        "age": عمر رقمي,
        "background": "خلفية مختصرة في جملة واحدة",
        "expertise": "مجال خبرته في جملة واحدة",
        "personality": "وصف شخصيته في جملة واحدة",
        "speaking_style": "أسلوب حديثه في جملة واحدة"
    }},
    "why_good_match": "لماذا هذا المقدم والضيف مناسبان لهذا الموضوع - جملة واحدة"
}}

متطلبات:
- أسماء عربية مألوفة
- شخصيات بسيطة وقابلة للتصديق
- مناسبة للموضوع والأسلوب المطلوب
- المقدم فضولي والضيف خبير أو صاحب تجربة
- لا تضع علامات ```json
- أرجع JSON فقط
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تصميم شخصيات بودكاست بسيطة ومؤثرة. الأسلوب المطلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.6
        )
        
        return response.choices[0].message.content

    def validate_personas(self, personas_json):
        """
        Simple validation for personas
        """
        required_keys = ["host", "guest", "why_good_match"]
        host_required = ["name", "age", "background", "personality", "speaking_style"]
        guest_required = ["name", "age", "background", "expertise", "personality", "speaking_style"]
        
        try:
            import json
            personas = json.loads(personas_json)
            
            missing_keys = []
            for key in required_keys:
                if key not in personas:
                    missing_keys.append(key)
            
            if "host" in personas:
                for key in host_required:
                    if key not in personas["host"]:
                        missing_keys.append(f"host.{key}")
            
            if "guest" in personas:
                for key in guest_required:
                    if key not in personas["guest"]:
                        missing_keys.append(f"guest.{key}")
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Personas validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

# Simple testing function
def test_simple_personas(deployment, classification_result):
    """
    Test simple persona generation
    """
    generator = SimplePersonaGenerator(deployment, "Fanar")
    
    topic = "ظاهرة العنوسة في المجتمع العربي: أسباب وحلول"
    information = """
    ظاهرة العنوسة تشهد ارتفاعاً ملحوظاً في المجتمعات العربية، حيث تشير الإحصائيات إلى أن 30% من النساء في دول الخليج 
    و 25% من الرجال تجاوزوا سن الثلاثين دون زواج، مقارنة بـ 15% قبل عقدين.
    """
    
    # Generate personas
    personas_result = generator.generate_personas(topic, information, classification_result)
    print("Generated Personas:")
    print(personas_result)
    
    # Validate
    is_valid, message = generator.validate_personas(personas_result)
    print(f"\nValidation: {message}")
    
    # Parse and show key info
    if is_valid:
        try:
            import json
            personas = json.loads(personas_result)
            print(f"\nHost: {personas['host']['name']} ({personas['host']['age']}) - {personas['host']['background']}")
            print(f"Guest: {personas['guest']['name']} ({personas['guest']['age']}) - {personas['guest']['expertise']}")
            print(f"Match Reason: {personas['why_good_match']}")
        except:
            print("Could not parse persona details")
    
    return personas_result

In [10]:
personas_result = test_simple_personas(deployment, classification_result)

Generated Personas:
{
    "host": {
        "name": "أحمد الشامي",
        "age": 32,
        "background": "صحفي متخصص في القضايا الاجتماعية في العالم العربي.",
        "personality": "فضولي ويحب استكشاف الجوانب المختلفة للظواهر الاجتماعية.",
        "speaking_style": "متوازن ويطرح أسئلة عميقة بطريقة بسيطة."
    },
    "guest": {
        "name": "د. ليلى السعدي",
        "age": 40,
        "background": "أستاذة علم الاجتماع في إحدى الجامعات العربية.",
        "expertise": "تبحث في تأثير التحولات الاقتصادية والثقافية على العلاقات الاجتماعية.",
        "personality": "هادئة ومحللة تقدم رؤى مدروسة.",
        "speaking_style": "منهجي ومبني على الأرقام والأبحاث مع أمثلة واضحة."
    },
    "why_good_match": "أحمد يطرح أسئلة تثير النقاش وليلى تقدم إجابات مدروسة تستند إلى خبرتها الأكاديمية."
}

Validation: Personas validation successful

Host: أحمد الشامي (32) - صحفي متخصص في القضايا الاجتماعية في العالم العربي.
Guest: د. ليلى السعدي (40) - تبحث في تأثير التحولات الاقتصادية والثقافية على العل

In [19]:
class ContentOutlineGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def generate_content_outline(self, topic, information, classification_result, personas_result):
        """
        Generate content-focused outline using classification and personas
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON string from Step 1 classification
            personas_result: JSON string from Step 2 personas
            
        Returns:
            JSON with structured content outline
        """
        
        # Parse inputs
        import json
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
        except:
            raise ValueError("Invalid JSON provided for classification or personas")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        discourse_pattern = classification.get("discourse_pattern", "")
        key_discussion_angles = classification.get("key_discussion_angles", [])
        natural_tension_points = classification.get("natural_tension_points", [])
        recommended_duration = classification.get("recommended_duration", "10 دقيقة")
        
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        host_background = personas.get("host", {}).get("background", "")
        guest_expertise = personas.get("guest", {}).get("expertise", "")
        
        prompt = f"""
أنت خبير في تصميم هيكل المحتوى للبودكاست العربي.

المهمة: أنشئ مخطط محتوى منطقي ومترابط لهذه الحلقة.

الموضوع: {topic}
المعلومات: {information}

السياق:
- الفئة: {primary_category}
- الأسلوب: {optimal_style}  
- نمط الخطاب: {discourse_pattern}
- المدة المقترحة: {recommended_duration}

الشخصيات:
- المقدم: {host_name} - {host_background}
- الضيف: {guest_name} - {guest_expertise}

زوايا النقاش المهمة: {key_discussion_angles}
نقاط التوتر الطبيعية: {natural_tension_points}

أرجع مخطط المحتوى بصيغة JSON:

{{
    "episode_info": {{
        "title": "عنوان الحلقة",
        "main_theme": "الموضوع الرئيسي في جملة واحدة",
        "target_duration": "المدة المتوقعة",
        "content_style": "نوع المحتوى المطلوب"
    }},
    "content_structure": {{
        "opening": {{
            "hook": "جملة افتتاحية جذابة محددة",
            "topic_introduction": "تقديم الموضوع بوضوح",
            "guest_introduction": "تقديم الضيف مع ربطه بالموضوع",
            "episode_roadmap": "خريطة ما سنناقشه في الحلقة"
        }},
        "main_sections": [
            {{
                "section_title": "عنوان القسم الأول",
                "main_points": [
                    "النقطة الأساسية الأولى",
                    "النقطة الأساسية الثانية", 
                    "النقطة الأساسية الثالثة"
                ],
                "key_questions": [
                    "سؤال رئيسي سيطرحه المقدم",
                    "سؤال متابعة مهم"
                ],
                "expected_insights": [
                    "رؤية متوقعة من الضيف",
                    "معلومة جديدة محتملة"
                ],
                "transition_to_next": "كيف الانتقال للقسم التالي"
            }},
            {{
                "section_title": "عنوان القسم الثاني", 
                "main_points": [
                    "النقطة الأساسية الأولى",
                    "النقطة الأساسية الثانية",
                    "النقطة الأساسية الثالثة"
                ],
                "key_questions": [
                    "سؤال رئيسي للقسم الثاني",
                    "سؤال استكشافي"
                ],
                "expected_insights": [
                    "رؤية متوقعة مختلفة",
                    "تحليل عميق"
                ],
                "transition_to_next": "كيف الانتقال للقسم التالي"
            }},
            {{
                "section_title": "عنوان القسم الثالث",
                "main_points": [
                    "النقطة الأساسية الأولى", 
                    "النقطة الأساسية الثانية",
                    "النقطة الأساسية الثالثة"
                ],
                "key_questions": [
                    "سؤال للقسم الثالث",
                    "سؤال تطبيقي"
                ],
                "expected_insights": [
                    "حلول أو توصيات",
                    "نصائح عملية"
                ],
                "transition_to_next": "كيف الانتقال للختام"
            }}
        ],
        "closing": {{
            "key_takeaways": [
                "الخلاصة الأولى المهمة",
                "الخلاصة الثانية المهمة", 
                "الخلاصة الثالثة المهمة"
            ],
            "final_message": "الرسالة الختامية الأساسية",
            "call_to_action": "دعوة للعمل أو التفكير",
            "next_steps": "خطوات يمكن للمستمع اتخاذها",
            "closing_gratitude": "شكر الضيف والمستمعين"
        }}
    }},
    "content_flow": {{
        "logical_progression": "كيف تتدفق الأفكار منطقياً",
        "narrative_arc": "القوس السردي للحلقة",
        "engagement_points": [
            "نقاط جذب انتباه المستمع",
            "لحظات تفاعل متوقعة"
        ],
        "complexity_progression": "كيف تتطور صعوبة المحتوى"
    }},
    "cultural_integration": {{
        "relevant_examples": [
            "أمثلة محلية مناسبة للموضوع",
            "حالات عربية ذات صلة"
        ],
        "cultural_sensitivity_notes": [
            "نقاط حساسة يجب مراعاتها",
            "توجيهات ثقافية"
        ],
        "regional_relevance": "كيف يرتبط الموضوع بالمنطقة العربية"
    }}
}}

متطلبات المحتوى:
- يجب أن يكون المحتوى مرتبطاً مباشرة بالموضوع: {topic}
- مناسب للأسلوب: {optimal_style}
- يراعي خلفية الشخصيات المحددة
- منطقي ومترابط ومناسب للمدة المقترحة
- يتضمن نقاط تفاعل طبيعية بين المقدم والضيف
- مناسب ثقافياً للجمهور العربي
- لا تضع علامات ```json 
- أرجع JSON صحيح فقط
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تصميم هيكل المحتوى للبودكاست العربي. تخصصك في بناء تدفق منطقي للأفكار يناسب الأسلوب: {optimal_style}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5  # Balanced for structured content
        )
        
        return response.choices[0].message.content

    def validate_content_outline(self, outline_json):
        """
        Validate the content outline structure
        """
        required_keys = ["episode_info", "content_structure", "content_flow", "cultural_integration"]
        
        content_structure_required = ["opening", "main_sections", "closing"]
        opening_required = ["hook", "topic_introduction", "guest_introduction", "episode_roadmap"]
        section_required = ["section_title", "main_points", "key_questions", "expected_insights"]
        closing_required = ["key_takeaways", "final_message", "call_to_action"]
        
        try:
            import json
            outline = json.loads(outline_json)
            
            missing_keys = []
            
            # Check main structure
            for key in required_keys:
                if key not in outline:
                    missing_keys.append(key)
            
            # Check content structure
            if "content_structure" in outline:
                content_struct = outline["content_structure"]
                for key in content_structure_required:
                    if key not in content_struct:
                        missing_keys.append(f"content_structure.{key}")
                
                # Check opening
                if "opening" in content_struct:
                    for key in opening_required:
                        if key not in content_struct["opening"]:
                            missing_keys.append(f"opening.{key}")
                
                # Check main sections
                if "main_sections" in content_struct:
                    sections = content_struct["main_sections"]
                    if not isinstance(sections, list) or len(sections) < 2:
                        missing_keys.append("main_sections (need at least 2 sections)")
                    else:
                        for i, section in enumerate(sections):
                            for key in section_required:
                                if key not in section:
                                    missing_keys.append(f"main_sections[{i}].{key}")
                
                # Check closing
                if "closing" in content_struct:
                    for key in closing_required:
                        if key not in content_struct["closing"]:
                            missing_keys.append(f"closing.{key}")
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Content outline validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

    def analyze_content_quality(self, outline_json):
        """
        Analyze the quality and coherence of the content outline
        """
        try:
            import json
            outline = json.loads(outline_json)
            
            analysis = {}
            
            # Check section count and balance
            main_sections = outline.get("content_structure", {}).get("main_sections", [])
            analysis["section_count"] = len(main_sections)
            analysis["balanced_sections"] = all(
                len(section.get("main_points", [])) >= 2 for section in main_sections
            )
            
            # Check content flow
            content_flow = outline.get("content_flow", {})
            analysis["has_narrative_arc"] = bool(content_flow.get("narrative_arc"))
            analysis["has_engagement_points"] = len(content_flow.get("engagement_points", [])) >= 2
            
            # Check cultural integration
            cultural = outline.get("cultural_integration", {})
            analysis["cultural_examples"] = len(cultural.get("relevant_examples", []))
            analysis["cultural_sensitivity"] = bool(cultural.get("cultural_sensitivity_notes"))
            
            # Overall quality score
            quality_indicators = [
                analysis["section_count"] >= 3,
                analysis["balanced_sections"],
                analysis["has_narrative_arc"], 
                analysis["has_engagement_points"],
                analysis["cultural_examples"] >= 2
            ]
            analysis["quality_score"] = sum(quality_indicators)
            analysis["quality_rating"] = "ممتاز" if analysis["quality_score"] >= 4 else "جيد" if analysis["quality_score"] >= 3 else "يحتاج تحسين"
            
            return analysis
            
        except:
            return {"error": "Could not analyze content quality"}



In [20]:
# Testing function
def test_content_outline(deployment, classification_result, personas_result):
    """
    Test content outline generation
    """
    generator = ContentOutlineGenerator(deployment, "Fanar")
    
    topic = "ظاهرة العنوسة في المجتمع العربي: أسباب وحلول"
    information = """
    ظاهرة العنوسة تشهد ارتفاعاً ملحوظاً في المجتمعات العربية، حيث تشير الإحصائيات إلى أن 30% من النساء في دول الخليج 
    و 25% من الرجال تجاوزوا سن الثلاثين دون زواج، مقارنة بـ 15% قبل عقدين.
    """
    
    # Generate content outline
    outline_result = generator.generate_content_outline(topic, information, classification_result, personas_result)
    print("Generated Content Outline:")
    print(outline_result)
    
    # Validate
    is_valid, message = generator.validate_content_outline(outline_result)
    print(f"\nValidation: {message}")
    
    # Analyze quality
    if is_valid:
        quality_analysis = generator.analyze_content_quality(outline_result)
        print(f"\nQuality Analysis:")
        for key, value in quality_analysis.items():
            print(f"  {key}: {value}")
    
    return outline_result

In [21]:
outline_result = test_content_outline(deployment, classification_result, personas_result)

Generated Content Outline:
{
    "episode_info": {
        "title": "ظاهرة العنوسة في المجتمع العربي: أسباب وحلول",
        "main_theme": "تحليل ظاهرة العنوسة في العالم العربي من حيث الأسباب والحلول الممكنة للتعامل معها.",
        "target_duration": "30-40 دقيقة",
        "content_style": "تحليلي بأسلوب ودي"
    },
    "content_structure": {
        "opening": {
            "hook": "هل تعلم أن معدلات العنوسة في العالم العربي تضاعفت خلال العقدين الماضيين؟ ما الذي يحدث؟",
            "topic_introduction": "اليوم سنتناول ظاهرة العنوسة، إحدى القضايا الاجتماعية التي أصبحت حديث الساعة في العالم العربي، وسنبحث الأسباب والحلول.",
            "guest_introduction": "معنا اليوم د. ليلى السعدي، الباحثة في العلاقات الاجتماعية وتأثير التحولات الاقتصادية والثقافية، لتساعدنا في فهم هذه الظاهرة بشكل أعمق.",
            "episode_roadmap": "سنبدأ بتحليل الأسباب الاقتصادية والاجتماعية وراء ارتفاع معدلات العنوسة، ثم نتحدث عن تأثير وسائل التواصل الاجتماعي، ونختم بمناقشة الحلول وقصص النجاح."
        },
     

In [22]:
class SimpleStyleEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def enhance_with_category_style(self, content_outline_result, classification_result):
        """
        Apply simple category-specific style to content outline
        
        Args:
            content_outline_result: JSON string from Step 3 content outline
            classification_result: JSON string from Step 1 classification
            
        Returns:
            Enhanced outline with category style applied
        """
        
        # Parse inputs
        import json
        try:
            content_outline = json.loads(content_outline_result)
            classification = json.loads(classification_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        primary_category = classification.get("primary_category", "")
        optimal_style = classification.get("optimal_style", "")
        
        prompt = f"""
أنت خبير في تطبيق أساليب مختلفة للبودكاست العربي.

المهمة: طبق أسلوب الفئة على المخطط الموجود.

الفئة: {primary_category}
الأسلوب: {optimal_style}

المخطط الحالي:
{content_outline_result}

إرشادات حسب الفئة:
- العلوم والتكنولوجيا: أضف شرح مبسط وأمثلة عملية
- السياسة والشؤون العامة: أضف وجهات نظر متعددة ونقاش متوازن  
- القضايا الاجتماعية: أضف قصص شخصية وتجارب حقيقية
- الرياضة والترفيه: أضف حماس وذكريات مثيرة
- التاريخ والثقافة: أضف أسلوب قصصي وربط بالحاضر

أرجع النتيجة بصيغة JSON:

{{
    "style_adjustments": {{
        "opening_style": "كيف تبدأ الحلقة بأسلوب الفئة",
        "question_style": "نوع الأسئلة المناسبة للفئة",
        "discussion_approach": "كيف نناقش بأسلوب الفئة",
        "closing_style": "كيف نختتم بأسلوب الفئة"
    }},
    "enhanced_sections": [
        {{
            "title": "عنوان القسم الأول محسن",
            "approach": "كيف نتناول هذا القسم بأسلوب الفئة",
            "key_questions": [
                "سؤال مناسب لأسلوب الفئة",
                "سؤال متابعة"
            ]
        }},
        {{
            "title": "عنوان القسم الثاني محسن", 
            "approach": "منهج الفئة لهذا القسم",
            "key_questions": [
                "سؤال للقسم الثاني",
                "سؤال تحليلي"
            ]
        }},
        {{
            "title": "عنوان القسم الثالث محسن",
            "approach": "نهج الفئة للختام", 
            "key_questions": [
                "سؤال ختامي",
                "سؤال للمستقبل"
            ]
        }}
    ]
}}

متطلبات:
- طبق أسلوب الفئة {primary_category} فقط
- اجعل التحسينات بسيطة ومباشرة
- ركز على الأسئلة والنهج وليس التفاصيل المعقدة
- لا تضع علامات ```json
- أرجع JSON فقط
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في تطبيق أساليب بسيطة للبودكاست. الفئة: {primary_category}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5
        )
        
        return response.choices[0].message.content

    def validate_style_enhancement(self, enhanced_outline_json):
        """
        Simple validation for style enhancement
        """
        required_keys = ["style_adjustments", "enhanced_sections"]
        style_required = ["opening_style", "question_style", "discussion_approach", "closing_style"]
        
        try:
            import json
            outline = json.loads(enhanced_outline_json)
            
            missing_keys = []
            
            for key in required_keys:
                if key not in outline:
                    missing_keys.append(key)
            
            if "style_adjustments" in outline:
                style_adj = outline["style_adjustments"]
                for key in style_required:
                    if key not in style_adj:
                        missing_keys.append(f"style_adjustments.{key}")
            
            if "enhanced_sections" in outline:
                sections = outline["enhanced_sections"]
                if not isinstance(sections, list) or len(sections) < 2:
                    missing_keys.append("enhanced_sections (need at least 2)")
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Style enhancement validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

# Simple testing function
def test_simple_style_enhancement(deployment, content_outline_result, classification_result):
    """
    Test simple style enhancement
    """
    enhancer = SimpleStyleEnhancer(deployment, "Fanar")
    
    # Generate style enhancement
    enhanced_result = enhancer.enhance_with_category_style(content_outline_result, classification_result)
    print("Enhanced Outline:")
    print(enhanced_result)
    
    # Validate
    is_valid, message = enhancer.validate_style_enhancement(enhanced_result)
    print(f"\nValidation: {message}")
    
    # Show key adjustments
    if is_valid:
        try:
            import json
            enhanced = json.loads(enhanced_result)
            style_adj = enhanced.get("style_adjustments", {})
            print(f"\nStyle Adjustments Applied:")
            print(f"Opening: {style_adj.get('opening_style', 'N/A')}")
            print(f"Questions: {style_adj.get('question_style', 'N/A')}")
            print(f"Discussion: {style_adj.get('discussion_approach', 'N/A')}")
        except:
            print("Could not parse style adjustments")
    
    return enhanced_result

In [23]:
enhanced_result = test_simple_style_enhancement(deployment, outline_result, classification_result)

Enhanced Outline:
{
    "style_adjustments": {
        "opening_style": "ابدأ الحلقة بقصة شخصية أو تجربة حقيقية لأحد الأفراد الذين يعانون من العنوسة لتجذب اهتمام المستمعين.",
        "question_style": "استخدم أسئلة تثير المشاعر وتدفع المستمعين للتفكير في تأثير العنوسة على حياتهم الشخصية والمجتمع.",
        "discussion_approach": "ناقش القضايا الاجتماعية من خلال قصص واقعية وتجارب حقيقية لأفراد واجهوا التحديات المرتبطة بالعنوسة.",
        "closing_style": "اختم الحلقة برسالة ملهمة وقصة نجاح حقيقية لتشجيع المستمعين على اتخاذ خطوات إيجابية."
    },
    "enhanced_sections": [
        {
            "title": "الأسباب الاقتصادية والاجتماعية وراء ارتفاع معدلات العنوسة",
            "approach": "ابدأ بمشاركة قصة شخصية لشخص يعاني من تكاليف الزواج المرتفعة وتأثيرها على حياته، ثم انتقل لتحليل الأسباب الاجتماعية والاقتصادية بشكل أعمق.",
            "key_questions": [
                "كيف أثرت الظروف الاقتصادية على حياة الأفراد الذين تأخروا في الزواج؟",
                "هل لديك قصة حقيقية توضح كيف وا

In [24]:
class FinalOutlineEnhancer:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model
        self.deployment = deployment

    def add_spontaneity_and_culture(self, topic, information, classification_result, personas_result, enhanced_outline_result):
        """
        Add spontaneous elements and cultural integration to create final outline
        
        Args:
            topic: Main topic of the podcast episode
            information: Background information about the topic
            classification_result: JSON from Step 1
            personas_result: JSON from Step 2  
            enhanced_outline_result: JSON from Step 4
            
        Returns:
            Final enhanced outline with spontaneity and cultural elements
        """
        
        # Parse inputs
        import json
        try:
            classification = json.loads(classification_result)
            personas = json.loads(personas_result)
            enhanced_outline = json.loads(enhanced_outline_result)
        except:
            raise ValueError("Invalid JSON provided")
        
        # Extract key info
        primary_category = classification.get("primary_category", "")
        cultural_sensitivity = classification.get("cultural_sensitivity_level", "")
        host_name = personas.get("host", {}).get("name", "المقدم")
        guest_name = personas.get("guest", {}).get("name", "الضيف")
        host_personality = personas.get("host", {}).get("personality", "")
        guest_personality = personas.get("guest", {}).get("personality", "")
        
        prompt = f"""
أنت خبير في إضافة الحيوية والأصالة الثقافية للبودكاست العربي.

المهمة: أضف عناصر التلقائية والثقافة العربية للمخطط النهائي.

الموضوع: {topic}
المعلومات: {information}

السياق:
- الفئة: {primary_category}
- مستوى الحساسية الثقافية: {cultural_sensitivity}
- المقدم: {host_name} - {host_personality}
- الضيف: {guest_name} - {guest_personality}

المخطط المحسن:
{enhanced_outline_result}

أضف العناصر التالية وأرجع النتيجة بصيغة JSON:

{{
    "final_outline": {{
        "opening_with_spontaneity": {{
            "natural_greeting": "ترحيب طبيعي يناسب شخصية المقدم",
            "spontaneous_hook": "بداية تلقائية جذابة",
            "cultural_opening": "افتتاحية تحمل لمسة ثقافية عربية",
            "personality_touch": "لمسة شخصية من المقدم"
        }},
        "enhanced_sections_with_culture": [
            {{
                "section_title": "عنوان القسم الأول نهائي",
                "natural_transitions": [
                    "انتقال طبيعي لبداية القسم",
                    "ربط تلقائي بالقسم السابق"
                ],
                "spontaneous_moments": [
                    "لحظة تفاعل تلقائي متوقعة",
                    "موقف قد يثير ضحكة أو دهشة"
                ],
                "cultural_references": [
                    "مثل أو حكمة عربية مناسبة",
                    "مرجع ثقافي محلي ذو صلة"
                ],
                "personality_interactions": [
                    "كيف سيتفاعل المقدم بشخصيته",
                    "رد فعل متوقع من الضيف"
                ]
            }},
            {{
                "section_title": "عنوان القسم الثاني نهائي",
                "natural_transitions": [
                    "انتقال للقسم الثاني",
                    "ربط منطقي"
                ],
                "spontaneous_moments": [
                    "لحظة تفاعل للقسم الثاني",
                    "موقف عفوي محتمل"
                ],
                "cultural_references": [
                    "مرجع ثقافي للقسم الثاني",
                    "تجربة عربية مشتركة"
                ],
                "personality_interactions": [
                    "تفاعل شخصيات في القسم الثاني",
                    "ديناميكية متوقعة"
                ]
            }},
            {{
                "section_title": "عنوان القسم الثالث نهائي",
                "natural_transitions": [
                    "انتقال للختام",
                    "تمهيد للخلاصة"
                ],
                "spontaneous_moments": [
                    "لحظة ختامية مؤثرة",
                    "تفاعل نهائي طبيعي"
                ],
                "cultural_references": [
                    "حكمة ختامية عربية",
                    "قول مأثور مناسب"
                ],
                "personality_interactions": [
                    "تفاعل ختامي للشخصيات",
                    "لمسة إنسانية أخيرة"
                ]
            }}
        ],
        "natural_closing": {{
            "spontaneous_conclusion": "ختام تلقائي وطبيعي",
            "cultural_farewell": "وداع يحمل لمسة ثقافية عربية",
            "personality_goodbye": "وداع شخصي من المقدم والضيف",
            "audience_connection": "ربط أخير مع الجمهور"
        }}
    }},
    "spontaneity_guide": {{
        "natural_fillers": [
            "حشو طبيعي مناسب للمقدم",
            "تعبيرات تلقائية للضيف"
        ],
        "reaction_triggers": [
            "ما قد يثير رد فعل طبيعي من المقدم",
            "ما قد يفاجئ الضيف ويجعله يتفاعل"
        ],
        "interruption_points": [
            "نقاط مقاطعة طبيعية محتملة",
            "لحظات تداخل إيجابي"
        ],
        "emotional_moments": [
            "لحظات قد تثير المشاعر",
            "نقاط تأثر طبيعية"
        ]
    }},
    "cultural_authenticity": {{
        "regional_touches": [
            "لمسات إقليمية مناسبة للموضوع",
            "مراجع محلية ذات صلة"
        ],
        "shared_experiences": [
            "تجارب عربية مشتركة",
            "ذكريات جماعية"
        ],
        "language_nuances": [
            "تنويعات لغوية خفيفة",
            "تعبيرات محلية مناسبة"
        ],
        "cultural_values": [
            "قيم عربية مرتبطة بالموضوع",
            "مبادئ ثقافية ذات صلة"
        ]
    }}
}}

متطلبات مهمة:
- كل العناصر يجب أن تكون مرتبطة بالموضوع: {topic}
- مناسبة لمستوى الحساسية الثقافية: {cultural_sensitivity}
- تعكس شخصيات المقدم والضيف المحددة
- طبيعية وغير مفتعلة
- أصيلة ثقافياً للجمهور العربي
- لا تضع علامات ```json
- أرجع JSON فقط
"""
        
        response = self.deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": f"أنت خبير في إضافة الحيوية والأصالة الثقافية للبودكاست العربي. الموضوع: {topic}"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7  # Higher creativity for spontaneity
        )
        
        return response.choices[0].message.content

    def validate_final_outline(self, final_outline_json):
        """
        Validate the final enhanced outline
        """
        required_keys = ["final_outline", "spontaneity_guide", "cultural_authenticity"]
        
        final_outline_required = ["opening_with_spontaneity", "enhanced_sections_with_culture", "natural_closing"]
        spontaneity_required = ["natural_fillers", "reaction_triggers", "emotional_moments"]
        cultural_required = ["regional_touches", "shared_experiences", "cultural_values"]
        
        try:
            import json
            outline = json.loads(final_outline_json)
            
            missing_keys = []
            
            # Check main structure
            for key in required_keys:
                if key not in outline:
                    missing_keys.append(key)
            
            # Check final outline
            if "final_outline" in outline:
                final_out = outline["final_outline"]
                for key in final_outline_required:
                    if key not in final_out:
                        missing_keys.append(f"final_outline.{key}")
                
                # Check sections
                if "enhanced_sections_with_culture" in final_out:
                    sections = final_out["enhanced_sections_with_culture"]
                    if not isinstance(sections, list) or len(sections) < 2:
                        missing_keys.append("enhanced_sections_with_culture (need at least 2)")
            
            # Check spontaneity guide
            if "spontaneity_guide" in outline:
                spont = outline["spontaneity_guide"]
                for key in spontaneity_required:
                    if key not in spont:
                        missing_keys.append(f"spontaneity_guide.{key}")
            
            # Check cultural authenticity
            if "cultural_authenticity" in outline:
                cultural = outline["cultural_authenticity"]
                for key in cultural_required:
                    if key not in cultural:
                        missing_keys.append(f"cultural_authenticity.{key}")
            
            if missing_keys:
                return False, f"Missing required keys: {missing_keys}"
            
            return True, "Final outline validation successful"
            
        except json.JSONDecodeError:
            return False, "Invalid JSON format"

    def analyze_outline_completeness(self, final_outline_json):
        """
        Analyze the completeness and quality of the final outline
        """
        try:
            import json
            outline = json.loads(final_outline_json)
            
            analysis = {}
            
            # Check spontaneity elements
            spontaneity = outline.get("spontaneity_guide", {})
            analysis["spontaneity_score"] = len([
                v for v in spontaneity.values() 
                if isinstance(v, list) and len(v) >= 2
            ])
            
            # Check cultural elements
            cultural = outline.get("cultural_authenticity", {})
            analysis["cultural_score"] = len([
                v for v in cultural.values()
                if isinstance(v, list) and len(v) >= 2
            ])
            
            # Check section completeness
            final_out = outline.get("final_outline", {})
            sections = final_out.get("enhanced_sections_with_culture", [])
            analysis["sections_complete"] = all(
                len(section.get("spontaneous_moments", [])) >= 1 and
                len(section.get("cultural_references", [])) >= 1
                for section in sections
            )
            
            # Overall readiness score
            readiness_indicators = [
                analysis["spontaneity_score"] >= 3,
                analysis["cultural_score"] >= 3,
                analysis["sections_complete"],
                bool(final_out.get("opening_with_spontaneity")),
                bool(final_out.get("natural_closing"))
            ]
            analysis["readiness_score"] = sum(readiness_indicators)
            analysis["ready_for_dialogue"] = analysis["readiness_score"] >= 4
            
            return analysis
            
        except:
            return {"error": "Could not analyze outline completeness"}

# Testing function
def test_final_enhancement(deployment, topic, information, classification_result, personas_result, enhanced_outline_result):
    """
    Test final enhancement with spontaneity and cultural integration
    """
    enhancer = FinalOutlineEnhancer(deployment, "Fanar")
    
    # Generate final enhancement
    final_result = enhancer.add_spontaneity_and_culture(
        topic, information, classification_result, personas_result, enhanced_outline_result
    )
    print("Final Enhanced Outline:")
    print(final_result)
    
    # Validate
    is_valid, message = enhancer.validate_final_outline(final_result)
    print(f"\nValidation: {message}")
    
    # Analyze completeness
    if is_valid:
        completeness = enhancer.analyze_outline_completeness(final_result)
        print(f"\nCompleteness Analysis:")
        for key, value in completeness.items():
            print(f"  {key}: {value}")
        
        if completeness.get("ready_for_dialogue", False):
            print("\n Outline is ready for dialogue generation!")
        else:
            print("\n Outline may need additional refinement")
    
    return final_result

In [25]:
final_result = test_final_enhancement(
    deployment, topic, information, 
    classification_result, personas_result, enhanced_result
)

Final Enhanced Outline:
{
    "final_outline": {
        "opening_with_spontaneity": {
            "natural_greeting": "أهلاً وسهلاً بكل مستمعينا الأعزاء في حلقة جديدة من بودكاستنا، اليوم موضوعنا حساس وقريب من قلوب الكثيرين.",
            "spontaneous_hook": "تخيلوا معي شخصاً تجاوز الثلاثين، ناجح في عمله، محبوب بين أصدقائه، لكنه يشعر وكأن شيئاً ما ينقصه... فما السبب؟",
            "cultural_opening": "في مجتمعاتنا العربية نقول دائماً 'الزواج نصف الدين'، لكن ماذا عن الأشخاص الذين تأخروا في تحقيق هذا النصف؟",
            "personality_touch": "أنا أحمد الشامي، أحب استكشاف القضايا الاجتماعية بشكل مختلف، واليوم سنغوص في موضوع العنوسة، مع ضيفتنا الرائعة د. ليلى السعدي."
        },
        "enhanced_sections_with_culture": [
            {
                "section_title": "الأسباب الاقتصادية والاجتماعية وراء ارتفاع معدلات العنوسة",
                "natural_transitions": [
                    "خلونا نبدأ بقصة حقيقية أثرت فيّ جداً، قصة عمر، شاب في الثلاثين من عمره، يحلم ببناء بيت صغير لكنه يواجه